# Ahead Of Time

<img src="https://github.com/lsteffenel/cours_numba_2017/blob/master/figures/numba_blue_icon_rgb.png?raw=1" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*8 novembre 2017*</center>

Nous avons vu jusqu'ici comment utiliser Numba en mode Just In Time. Si cette fonctionnalité est bien utile lorsque nous sommes dans une phase de développement, elle ne l'est pas forcément lorsque l'on veut faire un code de production. Vous pouvez bien évidemment utiliser l'option **cache=True** lorsque vous utilisez **@jit** pour que Numba ne recompile pas la fonction si elle est dans le cache. 

Mais imaginez que vous pouvez faire de l'Ahead Of Time, vous pourriez alors

- construire une fois pour toute vos kernels qui ne sont pas sensés évoluer vous épargnant ainsi le temps de compilation,
- diffuser directement votre code sans dépendances à Numba (la librairie est précompilée).

Néanmoins, cette fonctionnalité souffre pour le moment de quelques limitations

- vous ne pouvez pas l'utiliser sur des ufuncs;
- vous devez spécifier la signature de vos fonctions (les types acceptés en argument);
- chaque fonction exportée ne peut avoir qu'un signature;
- la compilation est faite avec l'architecture de votre machine. Alors que **@jit** optimise en fonction de l'architecture où le code tourne.

Numba offre deux manières d'utiliser l'Ahead Of Time

- soit directement dans un script Python,
- soit en utilisant distutils.

## Exemple dans un script

In [ ]:
from numba.pycc import CC

module = CC('addmod')

@module.export('add_double', 'f8(f8, f8)')
@module.export('add_int', 'i4(i4, i4)')
@module.export('add_array', 'f8[:](f8[:], f8[:])')
def add (a, b):
    return a + b

module.compile()

Une fois le module compilé, on peut l'importer et s'en servir.

In [ ]:
import addmod

print(addmod.add_double(1., 2.))
print(addmod.add_int(1, 2))

In [ ]:
import numpy as np

a = np.ones(10)
print(addmod.add_array(a, a))